In [1]:
import nltk
import json
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
with open ('BOT_CONFIG_new.json') as f:
    bot_config = json.load(f)
    

In [3]:
# text1 = 'мама мыла раму '
# text2 = 'корова шла и мама её остановила '
# text3 = 'ее раму украли и сломали '
# text4 = (text1+text2+text3).split()
# vocab = list((set(text4)))
# #vocab=['и', 'раму', 'ее', 'её', 'остановила', 'корова', 'украли', 'мыла', 'сломали', 'шла', 'мама']
# vector_of_text1 = [1,1,1,0,0,0,0,0,0]

In [3]:
X = []#все тексты которые есть
Y = []#все интенты всех текстов
for intent in bot_config['intents']:
    for example in bot_config['intents'][intent]['examples']:
        X.append(example)
        Y.append(intent)
vectorizer = CountVectorizer()
X_vecorized = vectorizer.fit_transform(X)
len(vectorizer.get_feature_names_out())
#print(X.toarray())
#643 текста и 643 вектора длиной 713
clf = RandomForestClassifier()
clf.fit(X_vecorized, Y)
clf.score(X_vecorized,Y)

0.8753894080996885

In [4]:
clf.predict(vectorizer.transform(['откуда ты']))

array(['whereyoufrom'], dtype='<U31')

In [5]:
def get_intent_by_model(text):
    return clf.predict(vectorizer.transform([text]))[0]
                        
def bot(text):
    intent = get_intent_by_model(text)
    return random.choice(bot_config['intents'][intent]['responses'])

input_text=''
while input_text !='stop':
    input_text = input()
    response = bot(input_text)
    print(response)


    

stop
может в лас вегас?


## Тестирование

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
X = []#все тексты которые есть
Y = []#все интенты всех текстов
for intent in bot_config['intents']:
    for example in bot_config['intents'][intent]['examples']:
        X.append(example)
        Y.append(intent)
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)
vectorizer = CountVectorizer(ngram_range=(1, 3),analyzer='char_wb',min_df=50)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
clf = LogisticRegression(C=10,max_iter=9000)
clf.fit(X_train_vectorized, y_train)
print(clf.score(X_train_vectorized,y_train))

print(clf.score(X_test_vectorized,y_test))



0.8908382066276803
0.14728682170542637


Интеграция с телеграмом

In [ ]:
#!/usr/bin/env python
# pylint: disable=C0116,W0613
# This program is dedicated to the public domain under the CC0 license.

"""
Simple Bot to reply to Telegram messages.

First, a few handler functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.

Usage:
Basic Echobot example, repeats messages.
Press Ctrl-C on the command line or send a signal to the process to stop the
bot.
"""

import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    update.message.reply_text(bot(update.message.text))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("2101904469:AAGpjZItUS8MgkslkA5Ij8Thz2U-T4bOu2A")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2021-11-16 16:19:31,974 - apscheduler.scheduler - INFO - Scheduler started


In [8]:
def clean(text):
    clean_text = ''
    for ch in text.lower():
        if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ':
            clean_text = clean_text + ch
    return clean_text
    
def get_intent(text):
    for intent in bot_config['intense'].keys():
        for example in bot_config['intense'][intent]['examples']:
            cleaned_example = clean(example)
            cleaned_text = clean(text)
            if nltk.edit_distance(cleaned_example,cleaned_text)/min(len(cleaned_example),len(cleaned_text)) < 0.4:
                return intent
    return 'not_found'

In [10]:
def clean(text):
    clean_text = ''
    for ch in text:
        if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя ':
            clean_text = clean_text + ch
        return clean_text
    

In [11]:
clean('привет!!')

'п'